In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import Model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dropout, Dense, Conv2D, MaxPooling2D, LeakyReLU , GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import os

C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Janmejai\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
train_dir = os.path.join('data', 'train')
val_dir = os.path.join('data', 'val')


In [6]:
mobile = MobileNetV2(weights = "imagenet", include_top = False, input_shape = (224,224,3))

for layer in mobile._layers:
    layer.trainable = False
x = mobile.output
x = Flatten(name = "flatten")(x)
x = Dropout(0.4)(x)
x = Dense(4, activation = 'softmax')(x)

model = Model(inputs=mobile.input, outputs = x)

# If you want to retrain : 
# from tensorflow.keras.models import load_model
# model = load_model('model_name.h5')

In [7]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 30,
                                   height_shift_range=0.3,
                                   width_shift_range= 0.3,
                                   horizontal_flip = True,
                                  zoom_range = 0.3,
                                   shear_range = 0.2,
                                   fill_mode = 'nearest'
                                )

valid_datagen = ImageDataGenerator(rescale = 1./255)

train_images = train_datagen.flow_from_directory(train_dir,
                                                target_size = (224,224),
                                                batch_size = 25,
                                                class_mode = 'categorical',
                                                )
val_images = valid_datagen.flow_from_directory(val_dir,
                                               target_size = (224,224),
                                               batch_size = 15,
                                               class_mode = 'categorical',
                                              )


Found 4190 images belonging to 4 classes.
Found 278 images belonging to 4 classes.


In [9]:
# model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics= ['accuracy'])
import tensorflow
initial_lr = 1e-3
loss = "categorical_crossentropy"
model.compile(tensorflow.keras.optimizers.Adam(lr=initial_lr),loss = loss, metrics=['accuracy'])


In [10]:
import datetime
import time
log_dir = "logs/fit_binary/" + 'mobilenetv2_ver_1.0'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
history = model.fit(train_images,
            steps_per_epoch = train_images.n//train_images.batch_size,
            epochs = 20,
            validation_data = val_images,
            validation_steps = val_images.n//val_images.batch_size,
            verbose = 1,
            callbacks=[tensorboard_callback])


Epoch 1/20
  1/167 [..............................] - ETA: 0s - loss: 2.0828 - accuracy: 0.4800WARNING:tensorflow:From C:\Users\Janmejai\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
167/167 [==============================] - 72s 432ms/step - loss: 2.5067 - accuracy: 0.7890 - val_loss: 1.0072 - val_accuracy: 0.8741
Epoch 2/20
167/167 [==============================] - 61s 366ms/step - loss: 1.8576 - accuracy: 0.8720 - val_loss: 0.7473 - val_accuracy: 0.8963
Epoch 3/20
167/167 [==============================] - 57s 339ms/step - loss: 1.7969 - accuracy: 0.8694 - val_loss: 1.9522 - val_accuracy: 0.8630
Epoch 4/20
167/167 [==============================] - 44s 263ms/step - loss: 2.4654 - accuracy: 0.8663 - val_loss: 1.0319 - val_accuracy: 0.8889
Epoch 5/20
167/167 [==============================]

In [12]:
import tensorflow
initial_lr = 1e-4
loss = "categorical_crossentropy"
model.compile(tensorflow.keras.optimizers.Adam(lr=initial_lr),loss = loss, metrics=['accuracy'])
history = model.fit(train_images,
            steps_per_epoch = train_images.n//train_images.batch_size,
            epochs = 10,
            validation_data = val_images,
            validation_steps = val_images.n//val_images.batch_size,
            verbose = 1,
            callbacks=[tensorboard_callback])

Epoch 1/10
167/167 [==============================] - 64s 384ms/step - loss: 1.4526 - accuracy: 0.9421 - val_loss: 0.4581 - val_accuracy: 0.9704
Epoch 2/10
167/167 [==============================] - 43s 258ms/step - loss: 1.3369 - accuracy: 0.9429 - val_loss: 0.4878 - val_accuracy: 0.9704
Epoch 3/10
167/167 [==============================] - 43s 257ms/step - loss: 1.4958 - accuracy: 0.9412 - val_loss: 0.5305 - val_accuracy: 0.9704
Epoch 4/10
167/167 [==============================] - 43s 255ms/step - loss: 1.4454 - accuracy: 0.9409 - val_loss: 0.3371 - val_accuracy: 0.9778
Epoch 5/10
167/167 [==============================] - 43s 259ms/step - loss: 1.2070 - accuracy: 0.9467 - val_loss: 0.3498 - val_accuracy: 0.9815
Epoch 6/10
167/167 [==============================] - 44s 266ms/step - loss: 1.3874 - accuracy: 0.9390 - val_loss: 0.2610 - val_accuracy: 0.9778
Epoch 7/10
167/167 [==============================] - 44s 263ms/step - loss: 1.4296 - accuracy: 0.9414 - val_loss: 0.4154 - val_ac

In [13]:
model.save('mobilenetv2_ver_1.0.h5')

In [14]:
model.save_weights('mobilenetv2_ver_1.0')

In [15]:
test_dir = os.path.join('data', 'test')

gen = valid_datagen.flow_from_directory(test_dir, target_size = (224,224), batch_size=10, class_mode = None, shuffle=False)

prob = model.predict(gen, 100)

Found 189 images belonging to 4 classes.


In [18]:
# !pip install gradio

# Gradio is an amazing tool to check your model.
# Running this cell will start a local server.
# You can then go the link and check on your images.
import gradio as gr
labels = ['None', 'Paper', 'Rock', 'Scissors']

def classify_image(inp):
    inp = inp.reshape((-1, 244, 244, 3))
    inp = tensorflow.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = model.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(4)}



image = gr.inputs.Image(shape=(244, 244, 3))
label = gr.outputs.Label(num_top_classes=4)

gr.Interface(fn=classify_image, inputs=image, outputs=label, capture_session=True).launch()

Running locally at: http://127.0.0.1:7861/


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x1f5e3474b48>,
 'http://127.0.0.1:7861/',
 None)